## CRNN vs RCNN

- CRNN/RCNN은 CNN개념과 RNN 개념을 하나로 연결해서 설계된 모델
- CRNN (Convolutional Recurrent Neural Network)
    - First CNN after RNN
- RCNN (Recurrent Convolutional Neural Network)
    - First RNN after CNN
    
## CRNN Paper
- (2015)An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition 

## CRNN의 구조적 특징
- (1) 기존은 부분적 학습 및 튜닝 -> End-to-End 학습 (전체를 한번에 학습하는것)
- (2) 문자와 같은 디테일한 어노테이션을 필요로 하지 않고, 단어와 같은 시퀀스 라벨에서 직접적으로 학습할 수 있다.
- (3) 임의의 길이의 시계열 데이터를 다루기 가능 (no character segmentation, horizontal scale normalization)
- (4) 사전 정의된 어휘집에 갇혀있지 않으며, 정의된 어휘집 이든 아니든 좋은 성과를 보여줌
- (5) 레이블 시퀀스를 생성 할 수 있는 RNN의 특성을 가지고 있음
- (6) 이미지 데이터에서 유익한 특성을 직접 학습하는 DCNN과 같은 특성을 가지고, 사전처리 단계를 필요로 하지 않음

<hr>



## 0. Introduction
- 과거 DCNN
    - 이미지에 존재하는 문자들을 따로따로 인식 및 하나로 합쳐서 결과물을 내는 방식
    - 개별문자 분리 힘듬
- 장점 및 의의
    - 왼쪽부터 순차적으로 이미지 인식 및 기존에 존재하는 단어와의 관계를 계산
    - 문자단위가 아닌 Word 단위로 학습 (Word를 잘게 분리하는거니까)
    - 시퀀스의 길이에 구애받지 않고 DCNN 보다 훨씬 적은 Parameter를 가짐
    - Preprocessing 및 Segmentation 등의 과정 불필요

## Network Architecture
- Convolutional layer
    - 각각의 input 이미지에서 자동으로 특징 시퀀스를 추출
    - 채널별로 RNN에 나눠서 입력

- Recurrent layers
    - CNN에서 출력된 각각의 특징 시퀀스의 각 프레임에 대해 예측을 수행하기 위해 recurrent network가 생성

- Transcription layer
    - RNN에 의한 각 프레임에 대한 예측을 라벨 시퀀스로 변환
    
![CRNN_Architecture](img/CRNN_Architecture.png)



## 1. Feature Sequence Extraction
- **이미지에서 Sequential 한 피처를 뽑아내는 것이 목적**
- Fully Connected Layer를 제외한 Convolution Layer, Maxpooling 만을 이용
- 모든 이미지는 같은 높이를 가지도록 크기가 조정 (각 열의 너비는 고정픽셀)
- 전체 이미지를 나타내는 Representation을 찾고자함
    - 그러려면 해당 Representation을 동일한 전처리 필요 및 해당 방법 어려움
- 특징 시퀀스내의 각 벡터는 그 지역에 대한 이미지 묘사임

![feature_sequence_extraction](img/feature_sequence_extraction.png)


<hr>

## 2. Sequence Labeling
- 순차적으로 예측하기 때문에, 왼쪽의 이미지에 대한 라벨로부터 영향을 받아 예측하게 됨 
     - 개별적으로 문자들을 독립적으로 인식하는 것보다 훨씬 안정적이고 효율적
- **쪼개서 순차적으로 인식을 하기 때문에, 이미지 전체의 라벨을 나타내야 하는 모델처럼 시퀀스의 길이를 지정해주지 않아도 됨**
- RNN은 역전파로 학습을 할 수 있기 때문에 Convolutional layer와 같이 학습을 반복 가능

- 전달되어지는 차이값의 시퀀스가 맵으로 연결 
- 피처맵을 피처시퀀스로 변환하는 작동을 반대로 진행 및 컨벌루션 레이어로 피드백
- CNN과 RNN사이의 다리 역할인 Map-to-Sequence라고 불리는 커스텀 레이어를 생성
- LSTM
    - RNN의 시퀀스가 길어지면 발생하는 그레디언트 소실문제 극복 위해 고안
    - memory cell, input gate, output gate, forget gate로 구성
        - memory cell은 과거의 문맥을 저장
        - input gate, output gate는 셀이 오랜시간동안 문맥을 저장 할 수 있게 함
        - 메모리에 있는 셀은 forget gate에 의해 삭제 될 수 있음
- Bidirectional LSTM
    - 기존 LSTM은 과거의 문맥만 사용하던 방법에서 양방향성으로 상호보완
    - 두개의 LSTM을 결합하여 앞->뒤, 뒤->앞 진행

![LSTM_BiLSTM](img/LSTM_BiLSTM.png)

<hr>


## 3. Transcription 
- RNN으로부터 만들어진 프레임당 예측 라벨값들을 시퀀스로 변환하는 과정

- 조각들을 합치면서 프레임당 예측이 가장 높을 확률과 라벨 시퀀스를 찾음
    - ex) 첫번째 조각엔 a,e,g 가 높음 -> 두세번째 조각 붙이면서 a로 확정

    
### 3-(1) Probability of label sequence
- 시퀀스의 길이가 T라고 하고, 그 T개만큼의 라벨이 주어졌을 때 공백으로 인식되는 부분을 지우고 존재하는 단어로의 확률값 계산 가능

### 3-(2) Transcription 두가지 방법
- 사전이 있는 lexicon-based transcriptions
- 사전이 존재하지 않는 lexicon-free

- Lexicon은 라벨 시퀀스의 집합
    - 가장 확률값을 크게하는 단어 얻는 것
    - 사전이 존재하지 않는 lexicon-free 모드
        - 예측은 어떤 lexicon없이 만들어짐.
        
    - 어휘사전 기반 lexicon-based transcriptions모드
        - 예측은 가장 높은 가능성을 가진 라벨 시퀀스의 선택으로 만들어짐.
    
    - 두 단계를 거치는 이유
        - 시퀀스 라벨에 대해 확률값을 계산하고 선택하는 시간 비용이 크므로
        - Lexicon-free 방식으로 추정을 어느정도 한 후 거리기반 가까운 확률의 라벨 선택
        

<hr>

## 참고문헌
- https://soyoung-new-challenge.tistory.com/5
- https://wujincheon.github.io/wujincheon.github.io/deep%20learning/2018/11/28/CRNN.html
